In [136]:
import pandas as pd 
from pathlib import Path
import alpaca_trade_api as tradeapi
import os 
from dotenv import load_dotenv
load_dotenv()
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca_trade_api.rest import REST, TimeFrame
import matplotlib.pyplot as plt
import seaborn as sns
from bear_class import BearMarket, YieldProcessor
import hvplot.pandas
import holoviews as hv

In [67]:
sp500_df = pd.read_csv(Path('../Bear-Market-Analysis/Resources/SPX 500 1972 --_ Present - Sheet1.csv'),
 infer_datetime_format= True, parse_dates=True)
sp500_df['Date'] = pd.to_datetime(sp500_df['Date']).dt.date
sp500_df = sp500_df.set_index('Date')
sp500_df = sp500_df.dropna()

#Now importing our 10yr yield data (ST Louis Fed):
ten_yr_yield_df = pd.read_csv(Path('../Bear-Market-Analysis//Resources/DGS10 (1).csv'), infer_datetime_format=True, index_col='DATE')
ten_yr_yield_df = ten_yr_yield_df.rename(columns={'DGS10':'yield'})
ten_yr_yield_df = ten_yr_yield_df.dropna()
ten_yr_yield_df.index.names = ['Date']

#2yr yield
two_yr_yield_df = pd.read_csv(Path('../Bear-Market-Analysis//Resources/DGS2 (1).csv'), infer_datetime_format=True, index_col = 'DATE')
two_yr_yield_df = two_yr_yield_df.rename(columns={'DGS2':'yield'})
two_yr_yield_df.dropna()
two_yr_yield_df.index.names = ['Date']

#because some responses have '.' in it for the yields, we must remove them to perform .pctchg():
ten_yr_yield_df = ten_yr_yield_df[ten_yr_yield_df['yield'] != '.']
two_yr_yield_df = two_yr_yield_df[two_yr_yield_df['yield'] != '.']
#change value to float 
two_yr_yield_df=two_yr_yield_df.apply(pd.to_numeric, errors='coerce')
ten_yr_yield_df=ten_yr_yield_df.apply(pd.to_numeric, errors='coerce')

#inflation plot to be used later
cpi = pd.read_csv(Path('../Bear-Market-Analysis//Resources/FPCPITOTLZGUSA.csv'),
 infer_datetime_format= True, parse_dates=True, index_col='DATE')

In [39]:
# Assuming sp500_df is defined elsewhere
processor = BearMarket(sp500_df)
processor.process_periods(['73_74', '80_82', '2000', '2008', '2020', '2022'])

bear_73_74, returns_73_74 = processor.get_data('73_74')
bear_80_82, returns_80_82 = processor.get_data('80_82')
bear_2000, returns_2000 = processor.get_data('2000')
bear_2008, returns_2008 = processor.get_data('2008')
bear_2020, returns_2020 = processor.get_data('2020')
bear_2022, returns_2022 = processor.get_data('2022')



In [40]:
# Assuming ten_yr_yield_df is defined elsewhere
processor = YieldProcessor(ten_yr_yield_df)
processor.process_periods(['73_74', '80_82', '2000', '2008', '2020', '2022'])

yield_73_74, yreturns_73_74 = processor.get_data('73_74')
yield_80_82, yreturns_80_82 = processor.get_data('80_82')
yield_2000, yreturns_2000 = processor.get_data('2000')
yield_2008, yreturns_2008 = processor.get_data('2008')
yield_2020, yreturns_2020 = processor.get_data('2020')
yield_2022, yreturns_2022 = processor.get_data('2022')


In [129]:
# List of labels and dataframes
labels = ['73_74', '80_82', '2000', '2008', '2020', '2022']
bear_returns_list = [returns_73_74, returns_80_82, returns_2000, returns_2008, returns_2020, returns_2022]
yield_returns_list = [yreturns_73_74, yreturns_80_82, yreturns_2000, yreturns_2008, yreturns_2020, yreturns_2022]


correlations = {}

for label, bear_ret, yield_ret in zip(labels, bear_returns_list, yield_returns_list):
    # Resetting the index
    yield_ret_reset = yield_ret.reset_index(drop=True)
    bear_ret_reset = bear_ret.reset_index(drop=True)

    # Concatenating side by side
    df = pd.concat([yield_ret_reset, bear_ret_reset], axis=1).dropna()

    # Getting correlation value from the corr matrix
    correlation = df.corr().iloc[0, 1]
    correlations[label] = correlation

correlation_df = pd.DataFrame(list(correlations.items()), columns=['Year_Period', 'Correlation'])
correlation_df.set_index('Year_Period', inplace=True)

In [130]:
correlation_df['Inflation'] = [cpi.loc["1973-01-01"]['FPCPITOTLZGUSA'],cpi.loc['1981-01-01']['FPCPITOTLZGUSA'],cpi.loc['2000-01-01']['FPCPITOTLZGUSA'],cpi.loc["2008-01-01"]['FPCPITOTLZGUSA'],cpi.loc['2020-01-01']['FPCPITOTLZGUSA'],cpi.loc["2021-01-01"]['FPCPITOTLZGUSA']]
correlation_df

,Correlation,Inflation
Year_Period,,
73_74,-0.038303,6.177760
80_82,-0.046396,10.334715
2000,-0.013659,3.376857
2008,-0.078719,3.839100
2020,0.522635,1.233584
2022,-0.080611,4.697859


In [131]:
# Capping the correlation values
threshold_max = 10
threshold_min = -10

correlation_df['Correlation'] = (-correlation_df['Correlation']*100).clip(threshold_min, threshold_max)


### Higher inflation environments lead to positive stock/bond correlation. 

In [132]:
#Correlation plot with stock/bond with inflation used as well
corr_plot = correlation_df.hvplot.bar(
    rot = 45,
    ylabel = 'Percent',
    xlabel = 'Six Analyzed Bear Markets',
    title = 'Inflations Effect on Stock/Bond Correlation',
    width=800,
    height=500,
    tools = ['hover'],
    legend= 'top_right',
    show_legend=True,
)
corr_plot

:Bars   [Year_Period,Variable]   (value)

### CPI:

In [163]:
import holoviews as hv

# Your original plot
cpi_plot = cpi.hvplot.line(
    ylabel='CPI',
    xlabel='Year',
    title='Inflation Rate from 1960-Present',
    frame_width=600,
    frame_height=450,
    grid=True,
    label="CPI"  # This sets the legend label for the main plot
)

# Create a horizontal line at 2% and label it "Target"
hline = hv.HLine(2).opts(color="red", line_width=1.5)
label = hv.Text(cpi.index[-42], 2.4, 'Target', halign='right').opts(color='red')

# Overlay the plots
combined_plot = (cpi_plot * hline * label)

combined_plot


:Overlay
   .Curve.CPI :Curve   [DATE]   (FPCPITOTLZGUSA)
   .HLine.I   :HLine   [x,y]
   .Text.I    :Text   [x,y]

In [173]:
merged_df = ten_yr_yield_df.merge(two_yr_yield_df, left_index=True, right_index=True, how='inner')
merged_df = merged_df.rename(columns={'yield_x': '10_yr', 'yield_y': '2_yr'})
inversion = merged_df['10_yr'] - merged_df['2_yr']
inversion.index = pd.to_datetime(inversion.index)

In [179]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

# Activate Bokeh for Jupyter Notebook
output_notebook()

# Create a new figure
p = figure(width=800, height=400, x_axis_type="datetime", title="Two Ten Inversion")

# Add a line to the figure
p.line(inversion.index, inversion, color='navy', legend_label='10yr - 2yr yields')

# Set the labels for axes
p.xaxis.axis_label = "Date"
p.yaxis.axis_label = "10yr - 2yr yields"

# Show the plot
show(p)


Loading BokehJS ...

In [180]:
#shows federal funds interest rates to be overset with CPI
fed_funds = pd.read_csv(Path('../Bear-Market-Analysis//Resources/FEDFUNDS.csv'),
 infer_datetime_format= True, parse_dates=True, index_col='DATE')

fed_funds_plot = fed_funds.hvplot.line()
fed_funds_plot

:Curve   [DATE]   (FEDFUNDS)

In [181]:
#create DF concatenating the cpi and fed funds to create a plot in future
funds_and_cpi = pd.concat([cpi,fed_funds], axis = 'columns', join= 'inner')
funds_and_cpi.columns= ['Inflation','FedFunds']
funds_and_cpi

,Inflation,FedFunds
DATE,,
1960-01-01,1.457976,3.99
1961-01-01,1.070724,1.45
1962-01-01,1.198773,2.15
1963-01-01,1.239669,2.92
1964-01-01,1.278912,3.48
...,...,...
2017-01-01,2.130110,0.65
2018-01-01,2.442583,1.41
2019-01-01,1.812210,2.40


In [182]:
#Fed funds overlaying cpi data
funds_and_cpi.hvplot.line(
    y=['Inflation','FedFunds'],
    x = 'DATE',
    ylabel = 'Inflation and Fed Funds Rate',
    xlabel = 'Year',
    legend = 'right'
    
)

:NdOverlay   [Variable]
   :Curve   [DATE]   (value)

In [189]:
#input a custom made csv that shows high counts for when dates inverted at yield curve to overlay with snp
invert_this = pd.read_csv(Path('../Bear-Market-Analysis//Resources/Untitled spreadsheet - Sheet1 (1).csv'),
 infer_datetime_format= True, parse_dates=True)
invert_this['Date'] = pd.to_datetime(invert_this['Date']).dt.date
invert_this = invert_this.set_index('Date')
invert_this = invert_this.dropna()
invert_this['Close'] = invert_this['Close'].astype('float64')

recess_detector = invert_this.hvplot.line()

In [191]:
#SNP dataplot to be used for overlay with yield inversion
snp_data_plot = sp500_df.hvplot.line(
)
snp_data_plot * recess_detector

:Overlay
   .Curve.I  :Curve   [Date]   (Close)
   .Curve.II :Curve   [Date]   (Close)